### Note: 
- Guardrails are a way to validate the input and output of an Agent to insure proper usage 

In [1]:
from agents import Agent, Runner, GuardrailFunctionOutput, RunContextWrapper, TResponseInputItem, input_guardrail, InputGuardrailTripwireTriggered, output_guardrail, OutputGuardrailTripwireTriggered
from dotenv import load_dotenv 
from pydantic import BaseModel
load_dotenv()

True

#### Input Guardrail

In [2]:
class HomeworkCheatDetectionOuput(BaseModel):
    attempting_cheat: bool
    explanation : str

homework_cheat_guardrail_agent = Agent(
    name="Homework Cheat Detector",
    instructions=(
        "Determine if the user's query resembles a tyical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable. "
        "Cheating: 'Fill in the blank: The capital of France is ___.'" 
        "'Solve for x: 2x + 3 = 7.' "
        "Which of the following best describe photosynthesis? A) A process by which plants make food. B) A type of animal behavior. C) A geological phenomenon."
        "Not cheating: 'Explain the theory of relativity.', 'What is the capital of France?'"
    ),
    output_type=HomeworkCheatDetectionOuput,
    model="gpt-4o-mini"
)

@input_guardrail
async def cheat_detection_guardrail(
        ctx:RunContextWrapper[None], agent:Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    
    detection_result = await Runner.run(homework_cheat_guardrail_agent, input=input)

    return GuardrailFunctionOutput(
        tripwire_triggered=detection_result.final_output.attempting_cheat,
        output_info=detection_result.final_output
    )


study_helper_agent = Agent(
    name = "Study Helper Agent",
    instructions="You assist user in studing by explaining concepts or providing guidance, without directly solving homework or test questions.",
    model="gpt-4o",
    input_guardrails=[cheat_detection_guardrail]
)

In [3]:
# This should raise an InputGuardrailTripwireTriggered exception
try: 
    response = await Runner.run(study_helper_agent, input="Fill in the blank: The capital of France is ___.")
    print("Guardrail didn't trigger")
    print("Response:", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Guardrail triggered!")
    # print("Tripwire Info:", e.tripwire_info)
    print("Exception details: ", str(e))

Guardrail triggered!
Exception details:  Guardrail InputGuardrail triggered tripwire


In [4]:
# This  will run successfully
try: 
    response = await Runner.run(study_helper_agent, input="What is the capital of France? and Why is it important?")
    print("Guardrail didn't trigger")
    print("Response:", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Guardrail triggered!")
    # print("Tripwire Info:", e.tripwire_info)
    print("Exception details: ", str(e))

Guardrail didn't trigger
Response: The capital of France is Paris. Paris is significant for several reasons:

1. **Cultural Influence**: It is a global center for art, fashion, gastronomy, and culture, home to iconic landmarks like the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

2. **Economic Importance**: As one of the largest European cities, Paris plays a key role in finance, commerce, and industry. It hosts numerous international businesses and organizations.

3. **Political Significance**: Paris is the political heart of France, where key governmental institutions are located, including the Élysée Palace, the residence of the French president.

4. **Historical Legacy**: The city has a rich history and has been a major European city for centuries, influencing politics, philosophy, and arts.

5. **Tourism**: It is one of the most visited cities globally, attracting millions of tourists annually.

The combination of these factors contributes to its prominence on the globa

#### Output Guardrail

In [5]:
class MessageOutput(BaseModel):
    response: str

@output_guardrail
async def forbidden_words_guardrails(ctx: RunContextWrapper, agent: Agent, output: str)->GuardrailFunctionOutput:
    print(f"Checking output for forbidden pharses: {output}")

    # Funny forbidden pharses to check
    forbidden_phrases = ["fart", "booger", "silly goose"]

    # Convert output to lowercase for case-insensitivitve comparison 
    output_lower = output.lower()

    # Check which forbidden phrases are present in the response
    found_phrases = [phrase for phrase in forbidden_phrases if phrase in output_lower]
    trip_triggered = bool(found_phrases)

    print(f"Found forbidden phrases: {found_phrases}")

    return GuardrailFunctionOutput(
        output_info={
            "reason": "Output contains forbidden phrases.",
            "forbidden_phrases_found": found_phrases,
        },
        tripwire_triggered=trip_triggered
    )


agent = Agent(
    name = "Customer support agent",
    instructions="You are a customer support agent, You help customer with their questions.",
    output_guardrails=[forbidden_words_guardrails],
    model="gpt-4o-mini",
)


In [7]:
# This  will run successfully
try: 
    await Runner.run(agent, "Say the word fart!")
    print("Guardrail didn't trip - this is unexpected")

except OutputGuardrailTripwireTriggered as e:
    print("The agent said a forbidden word! It is Fired!")

Checking output for forbidden pharses: Fart! If you have any questions or need assistance, feel free to ask!
Found forbidden phrases: ['fart']
The agent said a forbidden word! It is Fired!


In [8]:
# This  will run successfully
try: 
    await Runner.run(agent, "Hey there!")
    print("Guardrail didn't trip - this is unexpected")

except OutputGuardrailTripwireTriggered as e:
    print("The agent said a forbidden word! It is Fired!")

Checking output for forbidden pharses: Hi! How can I assist you today?
Found forbidden phrases: []
Guardrail didn't trip - this is unexpected
